# 1.4 스크래핑 프로세스
---

1. 웹페이지 크롤링
    - fetch(url) : 매개변수로 지정한 URL의 웹 페이지를 추출
2. 스크래핑
    - scrape(html) : 매개변수로 html을 받고, 정규 표현식을 사용해 HTML에서 도서 정보추출
3. 데이터 저장
    - save(db_path, books) : 매개변수로 books라는 도서 목록을 받고, db_path로 지정된 SQLite DB에 저장

### 2.7.1.1 python_scracper 실습

In [100]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [101]:
def main():
    """
    메인 처리입니다.
    fetch(), scrape(), save() 함수를 호출
    """
    html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')
    books= scrape(html)
    save('books.db', books)
    print('db생성완료')

In [102]:
# 웹페이지 크롤링 함수

def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    print(type(html))
    return html

In [124]:
# 웹페이지 스크래핑 (정규표현식 사용)
def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append({'url': url, 'title': title})
        print( re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL))
    return books

In [105]:
def save(db_path, books):
    """
    매개변수 books로 전달된 도서 목록을 SQLite 데이터베이스에 저장
    데이터베이스의 경로는 매개변수 db_path로 지정
    반환값:None()
    """
    
    # 데이터베이스를 열고 연결
    conn = sqlite3.connect(db_path)
    
    # 커서 추출
    c = conn.cursor()
    
    # execute() 메서드로 SQL을 실행
    # 스크립트를 여러번 실행할 수 있으므로 기존의 books 테이블 제거
    c.execute('DROP TABLE IF EXISTS books')
    
    # books 테이블을 생성
    c.execute('''
    CREATE TABLE books(
        title book,
        url text)
    ''')
    
    # executemany() 메서드를 사용하면 매개변수로 리스트를 지정
    c.executemany('INSERT INTO books VALUES (:title, :url)',books)
    
    # 변경사항을 커밋
    conn.commit()
    
    # 연결을 종료
    conn.close()

In [106]:
# db 출력
def db_select(db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            query = "SELECT * FROM {}".format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return df

In [107]:
main()

<class 'str'>
db생성완료


In [108]:
# db 출력
db_select('books.db','books').head()

,title,url
0,최신 관리회계,http://www.hanbit.co.kr/store/books/look.php?p...
1,리눅스 입문자를 위한 명령어 사전,http://www.hanbit.co.kr/store/books/look.php?p...
2,파타고니아 이야기,http://www.hanbit.co.kr/store/books/look.php?p...
3,"풀스택 서버리스 : 리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발",http://www.hanbit.co.kr/store/books/look.php?p...
4,한 권으로 배우는 작고 예쁜 꽃자수,http://www.hanbit.co.kr/store/books/look.php?p...


---

## HTML 스크래핑

- HTML 스크래핑을 위한 파이썬 라이브러리로 lxml, Beautiful Soup, pyquery 등이 존재
- lxml
    - XML 처리와 관련된 라이브러리인 libxml2와 libxslt의 파이썬 바인딩
    - 파이썬에서 사용이 용이하도록 API 구현
- Beautiful Soup
    - 간단하고 이해하기 쉬운 직관적인 API를 활용해 데이터 추출
    - 내부적으로 사용되는 파서를 목적에 맞게 변경

### lxml을 이용한 스크래핑
- lxml의 여러 API중 HTML을 파싱할 때는 lxml.html 사용
    - lxml.etree : ElementTree를 확장한 API를 가진 XML 파서
    - lxml.html : xml.etree를 기반으로 invalid HTML도 다룰 수 있게 해주는 HTML 파서
    - lxml.objectify : 트리를 객체 계층으로 다룰 수 있게 해주는 XML 파서 
    - lxml.sax : SAX 형식의 XML 파서

#### 3.3.2.1 scrape_by_lxml 실습

In [31]:
!pip install cssselect

In [38]:
import lxml.html

In [39]:
# HTML 파일을 읽어 들이고, getroot() 메서드로 HTMLElement 객체 생성
tree = lxml.html.parse('data/dp.html')
html = tree.getroot()

In [40]:
html

<Element html at 0x167742e35e0>

In [46]:
# cssselect() 메서드로 a 요소의 리스트를 추출 및 반복 수행
for a in html.cssselect('a'):
    # href 속성과 글자를 추출
    print(a.get('href'), a.text)

#id.gjdgxs 메뉴 바로가기
#id.30j0zll 검색 및 카테고리 바로가기
#id.1fob9te 본문 바로가기
https://www.google.com/url?q=https://www.hanbit.co.kr/index.html&sa=D&source=editors&ust=1626231380304000&usg=AOvVaw06nSbvRIHEzfmLTF53ezUS HOME
https://www.google.com/url?q=https://www.hanbit.co.kr/media/&sa=D&source=editors&ust=1626231380304000&usg=AOvVaw0SXpdReec57o7gJ1Ka90rE 한빛미디어
https://www.google.com/url?q=https://www.hanbit.co.kr/academy/&sa=D&source=editors&ust=1626231380305000&usg=AOvVaw2PLITYSqDn-qo4nLvIov3G 한빛아카데미
https://www.google.com/url?q=https://www.hanbit.co.kr/biz/&sa=D&source=editors&ust=1626231380305000&usg=AOvVaw1eyT3pz42el8C82_no8w2e 한빛비즈
https://www.google.com/url?q=https://www.hanbit.co.kr/life/&sa=D&source=editors&ust=1626231380306000&usg=AOvVaw2B6sxwO9aZExiSXfVQzr3E 한빛라이프
https://www.google.com/url?q=https://www.hanbit.co.kr/edu/&sa=D&source=editors&ust=1626231380306000&usg=AOvVaw2A2XjHkr1miLJGBRw2Sacp 한빛에듀
https://www.google.com/url?q=https://www.hanbit.co.kr/realtime/&sa=D&source=editors&ust=1

### Beautiful Soup을 이용한 스크래핑

| 파서 | 매개변수 | 특징 |
|---|:---:|---:|
| 표준라이브러리 html.parser | 'html.parser' | 추가 라이브러리 불필요 |
| lxml의 HTML 파서 | 'lxml' | 빠른 처리가 가능 |
| lxml의 XML 파서 | 'xml', 'lxml-xml' | XML에 대해 빠른 처리가 가능 |
| html5lib | 'html5lib' | html5lib을 사용해 HTML5의 사양에 맞게 파싱 가능 |


#### 3.3.2.2 scrape_by_bs4 실습

In [58]:
from bs4 import BeautifulSoup

In [83]:
# HTML 파일을 읽어 들이고 BeautifulSoup 객체를 생성
with open('data/full_book_list.html', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [84]:
# find_all() 메서드로 a 요소를 추출 및 반복 수행
for a in soup.find_all('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)

https://www.hanbit.co.kr/store/books/full_book_list.html#gnb 메뉴 바로가기
https://www.hanbit.co.kr/store/books/full_book_list.html#top_search 검색 및 카테고리 바로가기
https://www.hanbit.co.kr/store/books/full_book_list.html#container 본문 바로가기
https://www.hanbit.co.kr/index.html HOME
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.h